In [1]:
import pickle
import pandas as pd
from datetime import timedelta
import numpy as np

# 1. Import data

In [37]:
year=2050
nuc="nuclear_plus"
type_result="multinode_"#"" if singlenode
bati_hyp="ref"
reindus="UNIDEN"
with open("Results_"+type_result+str(year)+"_"+nuc+"_"+bati_hyp+"_"+reindus+".pickle", 'rb') as f:
    Variables=pickle.load(f)

In [14]:
print(Variables)

{'power_Dvar':        AREAS                TIME   TECHNOLOGIES    power_Dvar
0         IT 2019-10-07 22:00:00            TAC      0.000000
1         IT 2019-10-07 22:00:00  NewHydroRiver      0.000000
2         IT 2019-10-07 22:00:00   WindOffShore    331.000000
3         IT 2019-10-07 22:00:00     HydroRiver   6060.872585
4         IT 2019-10-07 22:00:00    curtailment      0.000000
...      ...                 ...            ...           ...
858475    ES 2019-04-21 17:00:00        Biomass      0.000000
858476    ES 2019-04-21 17:00:00           Coal      0.000000
858477    ES 2019-04-21 17:00:00    WindOnShore  11169.309928
858478    ES 2019-04-21 17:00:00        NewNuke      0.000000
858479    ES 2019-04-21 17:00:00        OldNuke      0.000000

[858480 rows x 4 columns], 'exchange_Pvar':        AREAS AREAS                TIME  exchange_Pvar
0         IT    IT 2019-10-07 22:00:00            0.0
1         IT    IT 2019-06-04 14:00:00            0.0
2         IT    IT 2019-04-02 12:0

# 2. Tools

## 2.1. Capacity

In [34]:
def prod_capa(area):
    df=Variables['capacity_Dvar']
    return df[df.AREAS==area].set_index('AREAS')[['TECHNOLOGIES','capacity_Dvar']]

def stock_capa(area):
    df1=Variables['Pmax_in_Dvar']
    df2=Variables['Pmax_out_Dvar']
    df3=Variables['Cmax_Pvar']
    df1['Pmax_out_Dvar']=df2['Pmax_out_Dvar']
    df1['Cmax_Pvar']=df3['Cmax_Pvar']
    df1['hour_stock']=
    return df1[df1.AREAS==area].set_index('AREAS')[['STOCK_TECHNO','Pmax_in_Dvar','Pmax_out_Dvar','Cmax_Pvar']]

# 3. Capacity 

In [38]:
# Production means capacity (MW)
prod_capa('FR')

,TECHNOLOGIES,capacity_Dvar
AREAS,,
FR,Solar,182039.181446
FR,curtailment,0.000000
FR,Coal,0.000000
FR,Geothermal,0.000000
FR,HydroReservoir,8785.000000
FR,WindOnShore,37000.000000
FR,HydroRiver,11030.000000
FR,Biomass,3000.000000
FR,TAC,0.000000


In [39]:
# Storage capacity
stock_capa('FR')
#Variables['Pmax_in_Dvar']

,STOCK_TECHNO,Pmax_in_Dvar,Pmax_out_Dvar,Cmax_Pvar
AREAS,,,,
FR,Battery,10832.234168,10832.234168,1.140235e+04
FR,PowertoH2toPower,24661.678021,0.000000,2.766081e+06
FR,HydroStorage,10899.000000,10899.000000,1.961820e+05


In [40]:
Variables['total_consumption_Pvar']['total_consumption_Pvar'].max()

156982.39567314554

In [41]:
Pow_df=Variables['power_Dvar']
Pow_df=Pow_df[Pow_df.AREAS=='FR']
Pow_df['power_Dvar'].sum()/1E6

844.8998137638106

In [42]:
Conso_df=Variables['total_consumption_Pvar']
Conso_df=Conso_df[Conso_df.AREAS=='FR']
Conso_df['total_consumption_Pvar'].sum()/1E6

717.2258681478339

In [43]:
P2H2_df=Variables['storageIn_Pvar']
P2H2_df=P2H2_df[(P2H2_df.STOCK_TECHNO=='PowertoH2toPower')&(P2H2_df.AREAS=='FR')]
P2H2_df['storageIn_Pvar'].sum()/1E6

128.35575017627718

In [44]:
H2P_df=Variables['storageOut_Pvar']
H2P_df=H2P_df[(H2P_df.STOCK_TECHNO=='PowertoH2toPower')&(P2H2_df.AREAS=='FR')]
H2P_df['storageOut_Pvar'].sum()/1E6

0.0

In [39]:
829.2152710940743-686.5192246872975-99.90221760406679+0.044936888363604995

42.838765691073576

In [73]:
# Storage means capacity (MW)
Variables['total_consumption_Pvar'].max()

TIME                      2019-12-31 23:00:00
total_consumption_Pvar          138364.610149
dtype: object

In [74]:
Variables['energy_Pvar'].max()

TIME           2019-12-31 23:00:00
energy_Pvar          138364.610149
dtype: object

In [7]:
df=Variables['power_Dvar']
print(df[df.TIME==pd.Timestamp('2019-12-31 23:00:00')])
print(df[df.TIME==pd.Timestamp('2019-12-31 23:00:00')]['power_Dvar'].sum())
df2=Variables['storageOut_Pvar']
print(df2[df2.TIME==pd.Timestamp('2019-12-31 23:00:00')])
df3=Variables['storageIn_Pvar']
print(df3[df3.TIME==pd.Timestamp('2019-12-31 23:00:00')])
df4=Variables['stockLevel_Pvar']
print(df4[df4.TIME==pd.Timestamp('2019-12-31 23:00:00')])
print(Variables['stockLevel_ini_Dvar'])

                     TIME    TECHNOLOGIES    power_Dvar
40989 2019-12-31 23:00:00     WindOnShore   4432.600000
40990 2019-12-31 23:00:00             CCG      0.000000
40991 2019-12-31 23:00:00         OldNuke  15520.000000
40992 2019-12-31 23:00:00             TAC      0.000000
40993 2019-12-31 23:00:00    WindOffShore  10598.700000
40994 2019-12-31 23:00:00  HydroReservoir   8785.000000
40995 2019-12-31 23:00:00         Biomass   3000.000000
40996 2019-12-31 23:00:00            Coal      0.000000
40997 2019-12-31 23:00:00           Solar      0.000000
40998 2019-12-31 23:00:00         NewNuke  21710.000000
40999 2019-12-31 23:00:00      HydroRiver   8233.891861
41000 2019-12-31 23:00:00   NewHydroRiver    895.799659
41001 2019-12-31 23:00:00     curtailment      0.000000
73175.99151963575
                    TIME      STOCK_TECHNO  storageOut_Pvar
9459 2019-12-31 23:00:00           Battery              0.0
9460 2019-12-31 23:00:00  PowertoH2toPower              0.0
9461 2019-12-31 23

In [8]:
df=Variables['power_Dvar']
df2=Variables['storageOut_Pvar']
df3=Variables['storageIn_Pvar']
df=df.groupby('TIME')['power_Dvar'].sum()
df2=df2.groupby('TIME')['storageOut_Pvar'].sum()
df3=df3.groupby('TIME')['storageIn_Pvar'].sum()
df_prod=pd.DataFrame()
df_prod['Prod']=df+df2-df3
#df_prod
#df_prod=df_prod.set_index('TIME')

df_diff=pd.DataFrame()
df_diff['diff']=df_prod['Prod']-Variables['total_consumption_Pvar'].set_index('TIME')['total_consumption_Pvar']
print(df_diff['diff'].max())
N_err=0
for i in df_diff.index:
    if np.abs(df_diff.loc[i,'diff'])>1:
        N_err+=1
        print(df_diff.loc[i,'diff'])
print(N_err)

4.3655745685100555e-11
0


In [9]:
df_prod.loc[pd.Timestamp('2019-12-31 23:00:00')]

Prod    73175.99152
Name: 2019-12-31 23:00:00, dtype: float64

In [10]:
Variables['total_consumption_Pvar'].set_index('TIME').loc[pd.Timestamp('2019-12-31 23:00:00')]

total_consumption_Pvar    73175.99152
Name: 2019-12-31 23:00:00, dtype: float64

In [21]:
Variables['total_consumption_Pvar'].max()

TIME                      2019-12-31 23:00:00
total_consumption_Pvar            155289.7205
dtype: object

In [12]:
Variables['total_consumption_Pvar'][Variables['total_consumption_Pvar'].total_consumption_Pvar==Variables['total_consumption_Pvar']['total_consumption_Pvar'].max()]

,TIME,total_consumption_Pvar
1290,2019-01-24 13:00:00,150448.942674


In [13]:
d0=pd.Timestamp('2019-01-01 00:00:00')
d1h=timedelta(hours=1)
df=Variables['flex_consumption_Pvar']
L_EV=[]
for h in range(24):
    L_EV.append(float(df[(df.TIME==d0+h*d1h)&(df.FLEX_CONSUM=='EV')]['flex_consumption_Pvar']))
L_EV

[8974.985765378075,
 11252.569669328535,
 10691.645219955992,
 11170.426644899537,
 13510.099229131103,
 15473.200406998334,
 16550.885092623164,
 4425.99954355268,
 5581.337750877695,
 20919.870411678494,
 21837.125896171616,
 23312.874833511825,
 28469.2682218418,
 25998.15287218346,
 27305.36161875611,
 25600.720420410304,
 20896.11392020079,
 18139.698630793904,
 17262.300474917676,
 15669.700657177991,
 15415.259850726845,
 17359.40017950799,
 25017.329324809274,
 23867.510910719506]

In [44]:
def Total_cost(Variables,area):
    return 1/1e9*(Variables['powerCosts_Pvar'][Variables['powerCosts_Pvar'].AREAS==area]['powerCosts_Pvar'].sum()\
            +Variables['capacityCosts_Pvar'][Variables['capacityCosts_Pvar'].AREAS==area]['capacityCosts_Pvar'].sum()\
            +Variables['storageCosts_Pvar'][Variables['storageCosts_Pvar'].AREAS==area]['storageCosts_Pvar'].sum()\
            +Variables['flexCosts_Pvar'][Variables['flexCosts_Pvar'].AREAS==area]['flexCosts_Pvar'].sum())
Total_cost(Variables,'FR')

39.551618023149395